In [0]:
!pip install wandb -q

     |████████████████████████████████| 1.4MB 9.2MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 460kB 44.5MB/s 
     |████████████████████████████████| 102kB 15.9MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 71kB 12.5MB/s 


In [0]:
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [0]:
#from keras.optimizers import SGD

In [0]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Dataset

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


In [0]:
%pwd

In [0]:
%ls

audio.wav             model_small.json              smalldata.rar
big_data_npy/         model_small_to_my_weights.h5  testdata/
CNN_model.json        npy_data/                     test_data_full/
CNN_model_weights.h5  npy_data.rar                  test_data_full.rar
data1.rar             path_to_my_weights.h5         testdata.rar
full_small_data_npy/  preprocess.py                 tfjsmodel/
kerasmodel            __pycache__/                  Train_data_full.rar
model_config.json     Small_data_npy/               wandb/


In [0]:
import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

In [0]:
wandb.init()
config = wandb.config

config.max_len = 11
config.buckets = 20
#max_len=11
#buckets=20

labels=['background noise', 'backward', 'bed', 'bird', 'cat', 'dog', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn',
        'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
indices=[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,34]


'''DATA_PATH = "./npy_data/"
# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


# convert file to wav2mfcc
# Mel-frequency cepstral coefficients
def wav2mfcc(file_path, n_mfcc=20, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = np.asfortranarray(wave[::3])
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=n_mfcc)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

def save_data_to_array(path=DATA_PATH, max_len=11, n_mfcc=20):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len, n_mfcc=n_mfcc)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

# Save data to array file first
save_data_to_array(max_len=max_len, n_mfcc=buckets)
'''

PATH = "./npy_data/"
def get_train_test(split_ratio=0.8, random_state=42):
    # Get available labels
    #labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    
    X = np.load(PATH+labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(PATH+label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)


# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
channels = 1
config.epochs = 50
config.batch_size = 32
num_classes = 35

X_train = X_train.reshape(X_train.shape[0], config.buckets, config.max_len)
X_test = X_test.reshape(X_test.shape[0], config.buckets, config.max_len)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)


#sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
#model.compile(loss="categorical_crossentropy",
#                  optimizer="sgd",
#                  metrics=['accuracy'])



In [0]:
# build model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16,(3,3),input_shape=(config.buckets, config.max_len,channels),
                        activation='relu',strides=(1,1),padding='same'))
model.add(layers.Conv2D(32,(3,3),activation='relu',strides=(1,1),padding='same'))
model.add(layers.Conv2D(64,(3,3),activation='relu',strides=(1,1),padding='same'))
model.add(layers.Conv2D(128,(3,3),activation='relu',strides=(1,1),padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

wandb.init()
model.fit(X_train, y_train_hot, epochs=config.epochs , batch_size=config.batch_size
          , validation_data=(X_test, y_test_hot),callbacks=[WandbCallback(data_type="image", labels=labels)])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 76094 samples, validate on 19024 samples
Epoch 1/50
76094/76094 [==============================] - 35s 458us/sample - loss: 2.4139 - acc: 0.3140 - val_loss: 1.2186 - val_acc: 0.6400
Epoch 2/50
76094/76094 [==============================] - 29s 382us/sample - loss: 1.3964 - acc: 0.5971 - val_loss: 0.9405 - val_acc: 0.7212
Epoch 3/50
76094/76094 [==============================] - 28s 368us/sample - loss: 1.1504 - acc: 0.6755 - val_loss: 0.8130 - val_acc: 0.7629
Epoch 4/50
76094/76094 [==============================] - 28s 372us/sample - loss: 1.0362 - acc: 0.7101 - val_loss: 0.7808 - val_acc: 0.7754
Epoch 5/50
76094/76094 [==============================] - 28s 363us/sample - loss: 0.9561 - acc: 0.7342 - val_loss: 0.7225 - val_acc: 0.7931
Epoch 6/50
76094/76094 [==============================] - 28s 373us/sample - loss: 0.9126 - acc: 0.7490 - val_loss: 0.7385 - val_acc: 0.7922
Epoch 7/50
76094/76094 [==============================] - 28s 362us/sample - loss: 0.8724 - acc: 0.7593 

In [0]:
# Save JSON config to disk
json_config = model.to_json()
with open('CNN_model.json', 'w') as json_file:
    json_file.write(json_config)
# Save weights to disk
model.save_weights('CNN_model_weights.h5')


In [0]:
# Reload the model from the 2 files we saved
with open('CNN_model.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('CNN_model_weights.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
labels_word = pd.Series(labels)
labels_indices = pd.Series(indices)
data=pd.DataFrame({ 'Total words': labels_word , 'Word indices': labels_indices })

In [0]:
%ls


big_data_npy/         model_small_to_my_weights.h5  testdata/
CNN_model.json        npy_data/                     test_data_full/
CNN_model_weights.h5  npy_data.rar                  test_data_full.rar
data1.rar             path_to_my_weights.h5         testdata.rar
full_small_data_npy/  preprocess.py                 tfjsmodel/
kerasmodel            __pycache__/                  Train_data_full.rar
model_config.json     Small_data_npy/               wandb/
model_small.json      smalldata.rar


In [0]:
audio_path='/content/drive/My Drive/Dataset/test_data_full/sheila/e2286c18_nohash_0.wav'
y,sr = librosa.load(audio_path,mono=True,sr=None)
wave = np.asfortranarray(y[::3])
#mfcc = librosa.feature.mfcc(wave)
mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=20)
if (11 > mfcc.shape[1]):
  pad_width = 11 - mfcc.shape[1]
  mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

# Else cutoff the remaining parts
else:
  mfcc = mfcc[:, :11]
Xnew = mfcc.reshape(1,20,11,1)
ynew = new_model.predict_classes(Xnew)
print(ynew)
data['Total words'][ynew]

[24]


24    sheila
Name: Total words, dtype: object

In [0]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 11, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 11, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 11, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 11, 128)       73856     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 5, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dropout (Dropout)            (None, 6400)              0

In [0]:
!pip -q install pydub
from IPython.display import Audio

In [0]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
#!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

'''def record(sec=2):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio'''

# all imports
'''from IPython.display import Javascript
from google.colab import output
from base64 import b64decode'''

def record(sec=1):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'

In [0]:
record()
Audio('audio.wav', autoplay=True)

<IPython.core.display.Javascript object>

In [0]:
!pwd

/content/drive/My Drive/Dataset


In [0]:
record()
audio_path='/content/drive/My Drive/Dataset/audio.wav'
y,sr = librosa.load(audio_path,mono=True,sr=None)
wave = np.asfortranarray(y[::3])
mfcc = librosa.feature.mfcc(wave,sr=16000, n_mfcc=20)
# If maximum length exceeds mfcc lengths then pad the remaining ones
if (11 > mfcc.shape[1]):
  pad_width = 11 - mfcc.shape[1]
  mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

# Else cutoff the remaining parts
else:
  mfcc = mfcc[:, :11]
Xnew = mfcc.reshape(1,20,11,1)
ynew = new_model.predict_classes(Xnew)
print(ynew)
data['Total words'][ynew]

<IPython.core.display.Javascript object>

[30]


30    up
Name: Total words, dtype: object

In [0]:
# build model
model = tf.keras.Sequential()
model.add(layers.LSTM(128,return_sequences=True,
                      input_shape=(config.buckets,config.max_len)))
model.add(layers.LSTM(128,return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(64,activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(32,activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(16,activation='relu')))
#model.add(layers.TimeDistributed(layers.Dense(8,activation='relu')))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

wandb.init()
model.fit(X_train, y_train_hot, epochs=config.epochs , batch_size=config.batch_size
          , validation_data=(X_test, y_test_hot),callbacks=[WandbCallback(data_type="image", labels=labels)])